## Python script to generate CRC for dsPIC33E flash

### Summary
The dsPIC uses 24bit (3bytes) wide instructions. The XC16 compiler generated HEX files are following INTEL byte standard and puts "phantom" 00 bytes to the 4th place. However the Microchip classB library CRC function *CLASSB_CRCFlashTest* does not calulate the phantom byte. (To save some calculation time.) Therefore most of the CRC calculators on the PC side just does not have proper configurability to give the same result as the ClassB library.

The below script calculates the CRC according the Microchip ClassB library implementation.

#### Readings:
* Microchip ClassB: https://www.microchip.com/design-centers/home-appliance/class-b-safety-software
* dsPIC33 architecture: https://microchipdeveloper.com/dsp0201:dspic-dsc-architecture-review
* INTEL HEX format: https://en.wikipedia.org/wiki/Intel_HEX

### Install dependencies
First install crcmod if needed:
* *pip install crcmod*
* *pip install intelhex*
### Contact
mark.wendler@microchip.com

Create CRC calculator function according to the default ClassB CRC calculation function. (Polynomal is 0x8005 by default in Microchip implementation)

In [1]:
import crcmod
crc16_func = crcmod.mkCrcFun(0x18005, rev=False, initCrc=0xFFFF, xorOut=0x0000)

Just a short test to verify crccalc functionality

In [2]:
word24bitIn = bytearray((0xe0, 0x00 , 0x00));

result = crc16_func(word24bitIn);

hex(result)

'0x380'

Install intelHex if needed: pip install intelhex

In [3]:
from intelhex import IntelHex16bit #Load intelHex
ih = IntelHex16bit("../input.hex") #Open file
ih.padding = 0xFF #change default padding
print("Max address:", hex(ih.maxaddr()))

Max address: 0xaffb


Create a function to extract hex file and format the data as crc calc requires.

In [4]:
def ExtractHexData(baseAddr, length, intelHex):
    """Rearrange memory map from intelHex to byte array: [Upper,Higher,Lower,Upper,Higher,Lower] This format can be used then for CRC calculation

    Args:
        baseAddr (int): Starting address to extract from hex file (dsPIC program counter address)
        length   (int): The length address of the tested flash memory in program counter units
        intelHex (IntelHex16bit): Extracted data from hex by the IntelHex module.
    Returns:
        bytearray: extraced byte array data, prepared for CRC calculation
    """    
    assert (baseAddr % 2) == 0,"Base address must be even. See dsPIC architecture program counter. See dsPIC architecture and classB lib CRC documentation."
    assert (length % 2) == 0,"Length must be specified as program counter's length. So must be multiple of 2. See dsPIC architecture and classB lib CRC documentation."
    assert (baseAddr + length <= ih.maxaddr()), "Address overflow. Check baseAddr, length and loaded hex file max address"
    
    byteLength = length + length//2 #Convert dsPIC program counts to byte count (24bit)
    print("ByteLength:", byteLength)
    crcInput = bytearray(byteLength) 
    byteAddr = 0;

    for addr in range(length): # Convert data according to the ClassB CRC implementation

        #check upper or higher,lower address [Upper,Higher,Lower => program word of dsPIC]
        if (addr % 2) == 0: #even means Higher and lower
            crcInput[byteAddr+1] = (intelHex[baseAddr + addr] >> 8 )& 0xFF;    #Get Higher
            crcInput[byteAddr+2] = intelHex[baseAddr + addr] & 0xFF; #Get Lower

            #print("Addr:", hex(baseAddr+addr), "Higher:", crcInput[byteAddr+1], "Lower:", crcInput[byteAddr+2])

        else: # odd addres mean Upper byte
            crcInput[byteAddr] = intelHex[baseAddr + addr]; 
            #print("Addr:", hex(baseAddr+addr), "Upper:", crcInput[byteAddr])
            byteAddr = byteAddr+3 #increase byteAddress as this was the 
        
    
    return crcInput

Finally calculate the CRC for the required range

In [5]:
crcInput = ExtractHexData(0x000,0xADE8,ih)
crcResult = crc16_func(crcInput)
print("CRC Result:", hex(crcResult))

ByteLength: 66780
CRC Result: 0xc12b


------------------ Utilities -----------------------------

In [6]:
# Find crcmod settings
counter = 1

def reverse_mask(x):
    x = ((x & 0x55555555) << 1) | ((x & 0xAAAAAAAA) >> 1)
    x = ((x & 0x33333333) << 2) | ((x & 0xCCCCCCCC) >> 2)
    x = ((x & 0x0F0F0F0F) << 4) | ((x & 0xF0F0F0F0) >> 4)
    x = ((x & 0x00FF00FF) << 8) | ((x & 0xFF00FF00) >> 8)
    x = ((x & 0x0000FFFF) << 16) | ((x & 0xFFFF0000) >> 16)
    return x


def showcrc(poly, start, reverse, xorval):
    start = start & 0xffffffff
    xorval = xorval & 0xffffffff
    reverse = bool(reverse)
    global counter
    crc_func = crcmod.mkCrcFun(poly, start, reverse, xorval)
    crc_res = crc_func(bytes(range(16)))
    print(f'{counter:2}) 0x{crc_res:08X}     \N{RIGHTWARDS ARROW}    crcmod.mkCrcFun(poly=0x{poly:08X}, initCrc=0x{start:08X}, rev={reverse}, xorOut=0x{xorval:08X})', end='')
    if crc_res == lookfor:
        print(f'  \N{check mark}')
    else:
        print()
    counter += 1


def iterate(poly, startval = 0, xorval = 0):
    showcrc(poly, startval, False, xorval)
    showcrc(poly, startval, False, ~xorval)
    showcrc(poly, startval, True, xorval)
    showcrc(poly, startval, True, ~xorval)
    showcrc(poly, ~startval, False, xorval)
    showcrc(poly, ~startval, False, ~xorval)
    showcrc(poly, ~startval, True, xorval)
    showcrc(poly, ~startval, True, ~xorval)

def iterate_both(poly, startval = 0, xorval = 0):
    print(f'Polygon: 0x{POLY:08X}')
    iterate(0x100000000 | POLY, startval, xorval)
    print(f'Refelcted polygon: 0x{reverse_mask(POLY):08X}')
    iterate(reverse_mask(POLY) | 0x100000000, startval, xorval)

POLY = 0x04C11DB7
lookfor = 0xCBC6DC44
print(f'Looking for result: {lookfor} for data 0...15: {list(range(16))}')
iterate_both(POLY, startval = 0x544F4F42)


Looking for result: 3418807364 for data 0...15: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Polygon: 0x04C11DB7
 1) 0x403093EF     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0x544F4F42, rev=False, xorOut=0x00000000)
 2) 0xEAE24ED8     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0x544F4F42, rev=False, xorOut=0xFFFFFFFF)
 3) 0xCBC6DC44     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0x544F4F42, rev=True, xorOut=0x00000000)  ✓
 4) 0x277D9711     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0x544F4F42, rev=True, xorOut=0xFFFFFFFF)
 5) 0x151DB127     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0xABB0B0BD, rev=False, xorOut=0x00000000)
 6) 0xBFCF6C10     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0xABB0B0BD, rev=False, xorOut=0xFFFFFFFF)
 7) 0xD88268EE     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0xABB0B0BD, rev=True, xorOut=0x00000000)
 8) 0x343923BB     →    crcmod.mkCrcFun(poly=0x104C11DB7, initCrc=0xABB0B0BD, rev=True, xorOut=0xFFFFFFFF)
Refelcted poly